In [1]:
import pandas as pd
import numpy as np
import py_vollib.black_scholes
import py_vollib_vectorized

grid definition

In [2]:
texp_dim=np.linspace(1,0,10)
spot_dim=np.arange(.05,2,.05)
[spot_grid,texp_grid]=np.meshgrid(spot_dim,texp_dim)

option definition

In [7]:
flag = 'c'  # 'c' for call, 'p' for put
K = 1  # Strikes
r=0
iv = 0.2  # Implied Volatilities
option_price=py_vollib.black_scholes.black_scholes(flag, spot_grid, K, texp_grid, r, iv,return_as='array')
option_greeks = py_vollib_vectorized.get_all_greeks(flag, 1, 1, .1, r, iv, return_as='dict')  

option_greeks['vega']


[0.0012609351106848202]

In [41]:
#90/110 risk reversal
cp =['p','c']
texp=[1,1]
r= 0
v = [0.2,0.2]  # Implied Volatilities
K = [0.9,1.1]  # Strikes
qty=[-100000000,100000000]
num_options=2

In [11]:
num_time_steps = 10
num_spots= 21
low_strike=.5   
high_strike=2

prepare for pricing

In [15]:
max_texp=np.max(texp)
texp_dim=np.linspace(max_texp,0,num_time_steps+1)[:-1]
spot_dim=np.linspace(low_strike,high_strike,num_spots)
[spot_grid,texp_grid]=np.meshgrid(spot_dim,texp_dim)

all_option_greeks={
    "delta":[],
    "gamma":[],
    "theta":[],
    "vega":[],
    "rho":[],
    "price":[]
}

for (qtyi,texpi,ki,cpi, ivi) in zip(qty,texp,K,cp,v):
    texp_gridi=texp_grid-(max_texp-texpi)
    texp_gridi_zero=texp_gridi.copy()
    texp_gridi_zero[texp_gridi_zero<0]=0
    option_price=py_vollib.black_scholes.black_scholes(cpi, spot_grid, ki, texp_gridi_zero, r, ivi,return_as='array')
    option_greeks = py_vollib_vectorized.get_all_greeks(cpi, spot_grid, ki, texp_gridi_zero, r, ivi, return_as='dict')
    gr_names=list(option_greeks.keys())
    option_greeks_arr={}
    for gr in gr_names:
        arr=np.array(option_greeks[gr])
        arr=arr.reshape(len(texp_dim),len(spot_dim))
        if(gr=="gamma"):
            arr=scale_gamma(arr,1)/100*spot_grid/1000000
        option_greeks_arr[gr]=arr
        all_option_greeks[gr].append(arr)
    arr=np.array(option_price).reshape(len(texp_dim),len(spot_dim))
    #arr=option_price.reshape(len(texp_dim),len(spot_dim))
    all_option_greeks["price"].append(arr)

portfolio_greeks={}
for gr in list(all_option_greeks.keys()):
    for (qtyi,i) in zip(qty,range(len(qty))):
        if portfolio_greeks.get(gr) is None:
            portfolio_greeks[gr]=qtyi*all_option_greeks[gr][i]
        else:
            portfolio_greeks[gr]=portfolio_greeks[gr]+qtyi*all_option_greeks[gr][i]


#for (gr, qtyi) in zip(list(all_option_greeks.keys()),qty):
#    portfolio_greeks[gr]=np.sum(np.array(all_option_greeks[gr]*qtyi),axis=0)


In [14]:
def scale_gamma(arr, spot):
    return arr*spot
    

In [274]:
#all_option_greeks.keys()
#portfolio_greeks["price"].shape
df = pd.DataFrame(portfolio_greeks["gamma"], columns = spot_dim, index = texp_dim)

def make_pretty(styler):
    styler.set_caption("Weather Conditions")
    styler.format(rain_condition)
    styler.format_index(lambda v: v.strftime("%A"))
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    return styler


df.style.format('{:.2f}').format_index('{:.2f}').format_index('{:.1%}', axis=1).background_gradient(axis=None, cmap="RdBu_r", low=.2, high=.2)

,50.0%,57.5%,65.0%,72.5%,80.0%,87.5%,95.0%,102.5%,110.0%,117.5%,125.0%,132.5%,140.0%,147.5%,155.0%,162.5%,170.0%,177.5%,185.0%,192.5%,200.0%
1.00,-0.03,-0.19,-0.54,-0.95,-1.11,-0.84,-0.23,0.43,0.90,1.10,1.08,0.92,0.71,0.51,0.35,0.22,0.14,0.08,0.05,0.03,0.02
0.90,-0.02,-0.15,-0.51,-0.99,-1.23,-0.97,-0.29,0.47,1.01,1.22,1.17,0.96,0.72,0.49,0.32,0.20,0.12,0.07,0.04,0.02,0.01
0.80,-0.01,-0.12,-0.46,-1.01,-1.36,-1.13,-0.36,0.53,1.15,1.37,1.26,1.00,0.71,0.46,0.28,0.16,0.09,0.05,0.02,0.01,0.01
0.70,-0.01,-0.08,-0.40,-1.02,-1.51,-1.33,-0.45,0.60,1.33,1.54,1.37,1.03,0.69,0.42,0.24,0.13,0.06,0.03,0.01,0.01,0.00
0.60,-0.00,-0.05,-0.32,-1.00,-1.67,-1.59,-0.59,0.70,1.56,1.76,1.48,1.04,0.64,0.36,0.18,0.09,0.04,0.02,0.01,0.00,0.00
0.50,-0.00,-0.02,-0.23,-0.93,-1.85,-1.94,-0.78,0.84,1.88,2.02,1.59,1.02,0.57,0.28,0.12,0.05,0.02,0.01,0.00,0.00,0.00
0.40,-0.00,-0.01,-0.14,-0.80,-2.01,-2.42,-1.07,1.05,2.34,2.36,1.68,0.95,0.45,0.18,0.07,0.02,0.01,0.00,0.00,0.00,0.00
0.30,-0.00,-0.00,-0.05,-0.58,-2.10,-3.10,-1.53,1.37,3.02,2.77,1.69,0.78,0.29,0.09,0.02,0.01,0.00,0.00,0.00,0.00,0.00
0.20,-0.00,-0.00,-0.01,-0.27,-1.98,-4.10,-2.36,1.92,4.13,3.24,1.50,0.47,0.10,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00
0.10,0.00,-0.00,-0.00,-0.02,-1.19,-5.77,-3.79,2.78,6.26,3.54,0.77,0.08,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,-0.00


In [262]:
df.columns.format('{:.2f}')

['',
 '0.500',
 '0.575',
 '0.650',
 '0.725',
 '0.800',
 '0.875',
 '0.950',
 '1.025',
 '1.100',
 '1.175',
 '1.250',
 '1.325',
 '1.400',
 '1.475',
 '1.550',
 '1.625',
 '1.700',
 '1.775',
 '1.850',
 '1.925',
 '2.000']

In [22]:
def create_adjusted_texp_grid(texp_grid, option_texp, max_texp):
    texp_gridi=texp_grid-(max_texp-option_texp)
    texp_gridi_zero=texp_gridi.copy()
    texp_gridi_zero[texp_gridi_zero<0]=0
    return texp_gridi_zero

def create_constant_grid(param, num_time_steps, num_spots):
    cp_grid=np.full((num_time_steps,num_spots),param, dtype=type(param))
    return cp_grid
#create_constant_grid(35.0, num_time_steps, num_spots)


In [23]:
max_texp=np.max(texp)
texp_dim=np.linspace(max_texp,0,num_time_steps+1)[:-1]
spot_dim=np.linspace(low_strike,high_strike,num_spots)
[spot_grid,texp_grid]=np.meshgrid(spot_dim,texp_dim)

In [55]:
pricing_grids=[]

for (qtyi,texpi,ki,cpi, ivi) in zip(qty,texp,K,cp,v):
    texp_gridi=create_adjusted_texp_grid(texp_grid, texpi, max_texp)
    cp_grid=create_constant_grid(cpi, num_time_steps, num_spots)
    iv_grid=create_constant_grid(ivi, num_time_steps, num_spots)
    ki_grid=create_constant_grid(ki, num_time_steps, num_spots)
    r_grid=create_constant_grid(r, num_time_steps, num_spots)
    qty_grid=create_constant_grid(qtyi, num_time_steps, num_spots)
    option_grids={
        "cp":cp_grid,
        "spot":spot_grid,
        "texp":texp_gridi,
        "iv":iv_grid,
        "ki":ki_grid,
        "r":r_grid,
        "qty":qty_grid
    }
    pricing_grids.append(option_grids)

# vstack the grids for each parameter into a single grid
inp_grids={}
for gr in list(pricing_grids[0].keys()):
    inp_grids[gr]=np.stack([pricing_grids[i][gr] for i in range(len(pricing_grids))])

option_price_c=py_vollib.black_scholes.black_scholes("c", inp_grids["spot"], inp_grids["ki"], inp_grids["texp"], inp_grids["r"], inp_grids["iv"],return_as='array')
option_greeks_c = py_vollib_vectorized.get_all_greeks("c", inp_grids["spot"], inp_grids["ki"], inp_grids["texp"], inp_grids["r"], inp_grids["iv"], return_as='dict')
option_price_p=py_vollib.black_scholes.black_scholes("p", inp_grids["spot"], inp_grids["ki"], inp_grids["texp"], inp_grids["r"], inp_grids["iv"],return_as='array')
option_greeks_p = py_vollib_vectorized.get_all_greeks("p", inp_grids["spot"], inp_grids["ki"], inp_grids["texp"], inp_grids["r"], inp_grids["iv"], return_as='dict')

option_price_c=option_price_c.reshape(num_options,num_time_steps,num_spots)
option_price_p=option_price_p.reshape(num_options,num_time_steps,num_spots)

for gr in list(option_greeks.keys()):
    option_greeks_c[gr]=np.array(option_greeks[gr]).reshape(num_options,num_time_steps,num_spots)
    option_greeks_p[gr]=np.array(option_greeks[gr]).reshape(num_options,num_time_steps,num_spots)


option_price=np.where(inp_grids["cp"]=="c",option_price_c,option_price_p)
option_greeks={}
for gr in list(option_greeks_c.keys()):
    option_greeks[gr]=np.where(inp_grids["cp"]=="c",option_greeks_c[gr],option_greeks_p[gr])


for gr in list(option_greeks.keys()):
    option_greeks[gr]=np.array(option_greeks[gr]).reshape(num_options,num_time_steps,num_spots)


option_greeks["price"]=option_price
option_greeks["delta"]*=inp_grids["qty"]

In [ ]:
class option_pricing_date:
    
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'